# **Modelo de predição Nexus**

In [6]:
!pip install -q pandas numpy matplotlib seaborn tqdm scikit-learn xgboost lightgbm gdown openpyxl rich
print('✅ Todas as bibliotecas foram instaladas com sucesso!')

✅ Todas as bibliotecas foram instaladas com sucesso!


In [7]:
# @title
# -*- coding: utf-8 -*-
"""Analise_Bioincrustacao_Frota_v2.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/github/bryanjulio/nexus_transpetro/blob/main/Analise_Bioincrustacao_Frota_v2.ipynb

# Predição de Bioincrustação Nexus
"""

import pkg_resources
import sys

def create_requirements_file(filename="requirements.txt"):
    # List of libraries explicitly used in the notebook
    explicit_dependencies = [
        'pandas',
        'numpy',
        'matplotlib',
        'seaborn',
        'tqdm',
        'scikit-learn',
        'xgboost',
        'lightgbm',
        'gdown', # Included if used for data download
        'openpyxl' # For reading/writing excel files, if applicable
    ]

    # Get all installed packages
    installed_packages = {p.project_name.lower(): p for p in pkg_resources.working_set}

    reqs = []
    for dep_name in explicit_dependencies:
        try:
            # Try to get the package, handling case-insensitivity
            package = installed_packages.get(dep_name.lower())
            if package:
                reqs.append(f"{package.project_name}=={package.version}")
            else:
                print(f"Warning: '{dep_name}' specified but not found in environment. Skipping.")
        except Exception as e:
            print(f"Error processing '{dep_name}': {e}. Skipping.")

    with open(filename, "w") as f:
        for r in sorted(reqs):
            f.write(r + "\n")
    print(f"Generated '{filename}' with {len(reqs)} key dependencies.")
    print("Please review the file and add any missing dependencies or remove unnecessary ones.")

create_requirements_file()

# import subprocess
# subprocess.run(["pip", "install", "-r", "requirements.txt"])

"""
Predição de Bioincrustação - Análise Avançada de Fouling


1. Features de tempo ocioso (idle time)
2. Features de velocidade de risco
3. Progressão temporal da bioincrustação
4. Validação temporal (não aleatória)
5. Modelo ensemble (XGBoost, LightGBM, RF, GB)
6. Target baseado em Fouling Rating IMO (0-4)
7. Análise de cenários futuros
8. Impacto econômico realista (5-25% penalty)
9. Análise individual por navio da frota
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
import zipfile
import os
warnings.filterwarnings('ignore')

# ML imports
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb
import pickle

# Rich imports para visualização
try:
    from rich.console import Console
    from rich.table import Table
    from rich.panel import Panel
    from rich.text import Text
    from rich import box
    RICH_AVAILABLE = True
    console = Console()
except ImportError:
    RICH_AVAILABLE = False
    print("⚠️ Rich não disponível. Usando visualização padrão.")
    print("   Instale com: pip install rich")

# Configurações
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

if RICH_AVAILABLE:
    console.print("[green]✓[/green] Bibliotecas importadas com sucesso!", style="bold")
else:
    print("✓ Bibliotecas importadas com sucesso!")

# Funções auxiliares para impressão limpa
def print_header(title):
    """Imprime cabeçalho de seção"""
    if RICH_AVAILABLE:
        console.print(f"\n[bold cyan]{title}[/bold cyan]")
        console.print("─" * len(title), style="cyan")
    else:
        print(f"\n{title}")
        print("=" * 80)

def print_info(message):
    """Imprime mensagem informativa"""
    if RICH_AVAILABLE:
        console.print(f"  {message}")
    else:
        print(f"  {message}")

def print_success(message):
    """Imprime mensagem de sucesso"""
    if RICH_AVAILABLE:
        console.print(f"[green]✓[/green] {message}")
    else:
        print(f"✓ {message}")

def print_warning(message):
    """Imprime aviso"""
    if RICH_AVAILABLE:
        console.print(f"[yellow]⚠[/yellow] {message}")
    else:
        print(f"⚠️ {message}")

def create_results_table(df_results, title="Resultados"):
    """Cria tabela formatada de resultados"""
    if not RICH_AVAILABLE:
        return df_results.to_string(index=False)

    table = Table(title=title, box=box.SIMPLE, show_header=True, header_style="bold")

    # Adicionar colunas
    for col in df_results.columns:
        table.add_column(col, justify="right" if df_results[col].dtype in ['int64', 'float64'] else "left")

    # Adicionar linhas
    for _, row in df_results.iterrows():
        table.add_row(*[str(val) for val in row])

    return table

import pkg_resources
import sys

def create_requirements_file(filename="requirements.txt"):
    # List of libraries explicitly used in the notebook
    explicit_dependencies = [
        'pandas',
        'numpy',
        'matplotlib',
        'seaborn',
        'tqdm',
        'scikit-learn',
        'xgboost',
        'lightgbm',
        'gdown', # Included if used for data download
        'openpyxl' # For reading/writing excel files, if applicable
    ]

    # Get all installed packages
    installed_packages = {p.project_name.lower(): p for p in pkg_resources.working_set}

    reqs = []
    for dep_name in explicit_dependencies:
        try:
            # Try to get the package, handling case-insensitivity
            package = installed_packages.get(dep_name.lower())
            if package:
                reqs.append(f"{package.project_name}=={package.version}")
            else:
                print(f"Warning: '{dep_name}' specified but not found in environment. Skipping.")
        except Exception as e:
            print(f"Error processing '{dep_name}': {e}. Skipping.")

    # Additionally, check for packages in the current environment that might be implied
    # This part is more general and might pick up extras, but ensures coverage
    # For a more precise list, manually curate explicit_dependencies.

    # Get all direct imports in the current kernel, though this is harder to automate perfectly.
    # For simplicity, we stick to explicit_dependencies here unless a more complex introspection is needed.

    with open(filename, "w") as f:
        for r in sorted(reqs):
            f.write(r + "\n")
    print(f"Generated '{filename}' with {len(reqs)} key dependencies.")
    print("Please review the file and add any missing dependencies or remove unnecessary ones.")

create_requirements_file()

Generated 'requirements.txt' with 10 key dependencies.
Please review the file and add any missing dependencies or remove unnecessary ones.


✓ Bibliotecas importadas com sucesso!

Generated 'requirements.txt' with 10 key dependencies.
Please review the file and add any missing dependencies or remove unnecessary ones.


## 1.5. DOWNLOAD DOS DADOS DO GOOGLE DRIVE (OPCIONAL)

In [8]:
# Se os dados não existirem localmente, baixar do Google Drive
DOWNLOAD_FROM_DRIVE = True  # Altere para True para baixar do Drive

if DOWNLOAD_FROM_DRIVE:
    try:
        import gdown
        print("\n Baixando dados do Google Drive...")

        folder_url = "https://drive.google.com/drive/folders/1NJrDlremklekCO1NR4Ltm43DZBOCKdW6"
        gdown.download_folder(folder_url, quiet=False, use_cookies=False, output="Hackathon Transpetro")

        print(" Dados baixados com sucesso!")
        BASE_PATH = "Dados Hackathon Transpetro/"
    except ImportError:
        print(" gdown não instalado. Execute: pip install gdown")
        print("   Usando dados locais...")
        BASE_PATH = ""
    except Exception as e:
        print(f" Erro ao baixar do Drive: {e}")
        print("   Usando dados locais...")
        BASE_PATH = ""
else:
    BASE_PATH = ""


 Baixando dados do Google Drive...


Retrieving folder contents


Retrieving folder 1oSLdQSsW0GpFgGoRZF12zFxFNs72P0sH Mais Dados
Processing file 1IzjTamdx1iq2MTi2VkrO6lAYF3s_uSIL AIS_NAVIO TESTE 2 1.csv
Processing file 1rP-GH7HLBMLS-DAQ6st9689wDtgY3YMe AIS_NAVIO TESTE 3 1.csv
Processing file 17PjAApZZCyk_2epri9x-CRyD-1BoUPor Consumo_Validacao 1.CSV
Processing file 1MoajA9gX0OHrEFdGyBRDh5DSXg0H7X-S Dados navios Validação 1.xlsx
Processing file 1xksJEcxznpN_anRavD_0x-4c2t8hQYRN Eventos_Validacao 1.CSV
Processing file 1BNU0xrGH54VYviSBNKGZG5Xkya4CTf0C RESULTADO Validação 1.xlsx
Processing file 17-kgs1RS52wenFcfpHr2ljbkq-4TG2Cf Dados AIS frota TP.zip
Processing file 1_CTM1V1PFN2guPl2ipW-VjdM80i8i7Ll Dados navios Hackathon.xlsx
Processing file 1L-iN3artAlSB3hqC6pQsX58z_9HKbmVw Dicionários de Dados.xlsx
Processing file 1YYp8B3finjq-p53MURKGyPXUQt26ZIuf Manual do Participante.pdf
Processing file 1ikd9AFsF18LZTAW8mRofQhVWaXzN98ek Relatorios IWS.xlsx
Processing file 1XUPl_mDEVjtlM6g19Q-oyaRWB9Gn6a0M ResultadoQueryConsumo.csv
Processing file 1S4iA70w2SapFOrNz1

Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1IzjTamdx1iq2MTi2VkrO6lAYF3s_uSIL
To: /content/Hackathon Transpetro/Mais Dados/AIS_NAVIO TESTE 2 1.csv
100%|██████████| 988k/988k [00:00<00:00, 3.46MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rP-GH7HLBMLS-DAQ6st9689wDtgY3YMe
To: /content/Hackathon Transpetro/Mais Dados/AIS_NAVIO TESTE 3 1.csv
100%|██████████| 1.22M/1.22M [00:00<00:00, 108MB/s]
Downloading...
From: https://drive.google.com/uc?id=17PjAApZZCyk_2epri9x-CRyD-1BoUPor
To: /content/Hackathon Transpetro/Mais Dados/Consumo_Validacao 1.CSV
100%|██████████| 191k/191k [00:00<00:00, 81.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1MoajA9gX0OHrEFdGyBRDh5DSXg0H7X-S
To: /content/Hackathon Transpetro/Mais Dados/Dados navios Validação 1.xlsx
100%|██████████| 12.9k/12.9k [00:00<00:00, 7.50MB/s]
Downloading...
From: https://drive.google.com/uc?id=1xksJEcxzn

 Dados baixados com sucesso!



Download completed


## 2. CARREGAMENTO DOS DADOS

In [9]:
# Se não foi definido BASE_PATH no download, usar caminho local
if 'BASE_PATH' not in locals():
    BASE_PATH = "/Users/bryan/Documents/Hackathon_transpetro/"

# Tentar múltiplos caminhos possíveis
data_paths = [
    BASE_PATH,
    "Hackathon Transpetro/",
    "/content/Hackathon Transpetro/",  # Google Colab
    ""  # Diretório atual
]

# Encontrar caminho válido
valid_path = None
for path in data_paths:
    if os.path.exists(f"{path}ResultadoQueryEventos.csv"):
        valid_path = path
        break

if valid_path is None:
    print("⚠️ Dados não encontrados. Configure DOWNLOAD_FROM_DRIVE=True ou ajuste BASE_PATH")
    exit(1)

BASE_PATH = valid_path
print(f"\n📂 Carregando dados de: {BASE_PATH}")

df_eventos = pd.read_csv(f"{BASE_PATH}ResultadoQueryEventos.csv")
df_consumo = pd.read_csv(f"{BASE_PATH}ResultadoQueryConsumo.csv")
df_navios = pd.read_excel(f"{BASE_PATH}Dados navios Hackathon.xlsx")
df_iws = pd.read_excel(f"{BASE_PATH}Relatorios IWS.xlsx")

print(f" Eventos: {df_eventos.shape}")
print(f" Consumo: {df_consumo.shape}")
print(f" Navios: {df_navios.shape}")
print(f" IWS: {df_iws.shape}")

# Carregar AIS
# Tentar múltiplos caminhos possíveis
ais_paths = [
    f"{BASE_PATH}Dados AIS frota TP",  # Pasta descompactada
    f"{BASE_PATH}notebooks/Dados Hackathon Transpetro/Dados AIS frota TP.zip",  # ZIP no notebooks
    f"{BASE_PATH}Dados AIS frota TP.zip"  # ZIP na raiz
]

df_ais = pd.DataFrame()
ais_loaded = False

for ais_path in ais_paths:
    if os.path.exists(ais_path):
        if ais_path.endswith('.zip'):
            # É um ZIP, extrair
            extract_folder = "dados_ais_temp"
            with zipfile.ZipFile(ais_path, "r") as z:
                z.extractall(extract_folder)
            csv_folder = os.path.join(extract_folder, "Dados AIS frota TP")
        else:
            # Já é uma pasta
            csv_folder = ais_path

        # Ler CSVs
        all_dfs = []
        for file_name in os.listdir(csv_folder):
            if file_name.lower().endswith(".csv"):
                file_path = os.path.join(csv_folder, file_name)
                df = pd.read_csv(file_path)
                df["ARQUIVO_ORIGEM"] = file_name
                all_dfs.append(df)

        if all_dfs:
            df_ais = pd.concat(all_dfs, ignore_index=True)
            print(f"✅ AIS carregado de {ais_path}: {df_ais.shape}")
            ais_loaded = True
            break

if not ais_loaded:
    print(" Arquivo AIS não encontrado em nenhum caminho")
    print(f"   Tentou: {ais_paths}")


📂 Carregando dados de: Hackathon Transpetro/
 Eventos: (50904, 22)
 Consumo: (87737, 3)
 Navios: (21, 8)
 IWS: (29, 14)
✅ AIS carregado de Hackathon Transpetro/Dados AIS frota TP.zip: (415724, 7)


2.5 Pré Processamento


In [10]:
# Padronizar colunas
df_eventos.columns = df_eventos.columns.str.strip()
df_consumo.columns = df_consumo.columns.str.strip()
df_navios.columns = df_navios.columns.str.strip()
df_iws.columns = df_iws.columns.str.strip()

# Parse datetimes
for c in ["startGMTDate", "endGMTDate"]:
    if c in df_eventos.columns:
        df_eventos[c] = pd.to_datetime(df_eventos[c], errors='coerce')

# Renomear SESSION_ID
if "SESSION_ID" in df_consumo.columns:
    df_consumo.rename(columns={"SESSION_ID": "sessionId"}, inplace=True)

# Processar AIS
if not df_ais.empty:
    df_ais.columns = df_ais.columns.str.strip()

    for cand in ["DATAHORA", "DataHora", "datahora", "DATETIME"]:
        if cand in df_ais.columns:
            df_ais['DATETIME'] = pd.to_datetime(df_ais[cand], errors='coerce')
            break

    for vcol in ["VELOCIDADE", "speed", "SOG", "speedGps"]:
        if vcol in df_ais.columns:
            df_ais['speed_kn'] = pd.to_numeric(df_ais[vcol], errors='coerce')
            break

    for cand in ["NOME", "name", "ship", "shipName", "ARQUIVO_ORIGEM"]:
        if cand in df_ais.columns:
            df_ais['shipName_ais'] = df_ais[cand].astype(str)
            break

## 3. AGREGAÇÃO AIS POR EVENTO

In [11]:
def aggregate_ais_by_event(df_eventos, df_ais):
    """Agrega dados AIS para cada evento de navegação"""
    agg_rows = []

    if df_eventos.empty or df_ais.empty:
        return pd.DataFrame()

    df_ais['shipName_ais_low'] = df_ais['shipName_ais'].str.lower().str.strip()
    df_eventos['shipName_low'] = df_eventos['shipName'].astype(str).str.lower().str.strip()

    ais_groups = {k: g for k, g in df_ais.groupby('shipName_ais_low')}

    for idx, ev in tqdm(df_eventos.iterrows(), total=len(df_eventos), desc="Agregando AIS"):
        ship = str(ev.get('shipName_low', "")).strip()
        sdt = ev.get('startGMTDate')
        edt = ev.get('endGMTDate')

        if ship == "" or pd.isna(sdt) or pd.isna(edt):
            continue

        ais_g = ais_groups.get(ship)
        if ais_g is None:
            candidates = [k for k in ais_groups.keys() if ship in k or k in ship]
            ais_g = ais_groups.get(candidates[0]) if candidates else None

        if ais_g is None:
            continue

        window = ais_g[(ais_g['DATETIME'] >= sdt) & (ais_g['DATETIME'] <= edt)]

        if window.empty:
            continue

        speed_mean = window['speed_kn'].mean()
        speed_std = window['speed_kn'].std()
        speed_min = window['speed_kn'].min()
        speed_max = window['speed_kn'].max()
        frac_stop = (window['speed_kn'] < 1.5).mean()
        frac_low_speed = (window['speed_kn'] < 5).mean()

        lat_mean = pd.to_numeric(window.get('LATITUDE', window.get('latitude', pd.Series(np.nan))), errors='coerce').mean()
        lon_mean = pd.to_numeric(window.get('LONGITUDE', window.get('longitude', pd.Series(np.nan))), errors='coerce').mean()

        agg_rows.append({
            'sessionId': ev.get('sessionId'),
            'shipName': ev.get('shipName'),
            'startGMTDate': sdt,
            'endGMTDate': edt,
            'duration_h': ev.get('duration'),
            'distance': ev.get('distance'),
            'beaufort': ev.get('beaufortScale'),
            'seaCondition': ev.get('seaCondition'),
            'displacement': ev.get('displacement'),
            'speed_mean': speed_mean,
            'speed_std': speed_std,
            'speed_min': speed_min,
            'speed_max': speed_max,
            'frac_stop': frac_stop,
            'frac_low_speed': frac_low_speed,
            'lat_mean': lat_mean,
            'lon_mean': lon_mean
        })

    return pd.DataFrame(agg_rows)

print("\n Agregando dados AIS...")
df_events_ais = aggregate_ais_by_event(df_eventos, df_ais)
print(f"Eventos com AIS: {df_events_ais.shape}")


 Agregando dados AIS...


Agregando AIS: 100%|██████████| 50904/50904 [00:44<00:00, 1154.24it/s]

Eventos com AIS: (8214, 17)


## 4. FEATURES AVANÇADAS

In [12]:
def create_advanced_features(df):
    """Cria features avançadas baseadas em ciência de bioincrustação"""
    df = df.copy()

    # 1. IDLE TIME FEATURES (CRÍTICO)
    df['idle_time_ratio'] = df['frac_stop'].fillna(0)
    df['idle_days'] = (df['duration_h'] * df['idle_time_ratio'] / 24).fillna(0)
    df['low_speed_days'] = (df['duration_h'] * df['frac_low_speed'] / 24).fillna(0)

    # 2. VELOCITY RISK SCORE (CRÍTICO)
    def velocity_risk(speed):
        if pd.isna(speed):
            return 2
        if speed < 5:
            return 3  # Alto risco
        elif speed < 10:
            return 2  # Risco moderado
        elif speed < 12:
            return 1  # Baixo-moderado
        else:
            return 0  # Baixo risco

    df['velocity_risk'] = df['speed_mean'].apply(velocity_risk)

    # 3. OPERATIONAL PROFILE
    df['operation_continuity'] = 1 - df['idle_time_ratio']

    # 4. LOW SHEAR ZONES EXPOSURE
    df['low_shear_exposure'] = df['idle_days'] * (df['velocity_risk'] + 1)

    # 5. BIOGEOGRAPHIC REGION RISK
    def get_biogeographic_region(lat):
        if pd.isna(lat):
            return 'Unknown'
        if lat > -5:
            return 'Norte'
        elif lat > -15:
            return 'Nordeste'
        else:
            return 'Sudeste-Sul'

    df['bio_region'] = df['lat_mean'].apply(get_biogeographic_region)
    region_risk = {'Norte': 3, 'Nordeste': 2, 'Sudeste-Sul': 1, 'Unknown': 1.5}
    df['region_risk'] = df['bio_region'].map(region_risk)

    # 6. TEMPERATURE PROXY
    df['temp_proxy'] = df['lat_mean'].abs().fillna(15)
    df['temp_risk'] = (15 - df['temp_proxy']).clip(0, 15) / 15

    # 7. SPEED VARIABILITY
    df['speed_variability'] = df['speed_std'] / (df['speed_mean'] + 1)

    return df

if not df_events_ais.empty:
    df_events_ais = create_advanced_features(df_events_ais)

## 5. PROCESSAR IWS E CRIAR TARGET

In [13]:
def process_iws_and_docking_data(df_iws, df_eventos, df_events_ais):
    """
    Processa dados de inspeção IWS E eventos de DOCAGEM
    para calcular dias desde última limpeza
    """
    if df_events_ais.empty:
        return df_events_ais

    cleaning_events = pd.DataFrame()

    # 1. Extrair datas de limpeza do IWS
    if not df_iws.empty:
        iw_cols = [c for c in df_iws.columns if 'data' in c.lower()]
        ship_cols = [c for c in df_iws.columns if 'embarca' in c.lower() or 'navio' in c.lower()]

        if iw_cols and ship_cols:
            date_col = iw_cols[0]
            ship_col = ship_cols[0]

            df_iws_clean = df_iws.copy()
            df_iws_clean['date_clean'] = pd.to_datetime(df_iws_clean[date_col], errors='coerce')
            df_iws_clean['ship_clean'] = df_iws_clean[ship_col].astype(str).str.lower().str.strip()
            df_iws_clean['source'] = 'IWS'

            cleaning_events = pd.concat([
                cleaning_events,
                df_iws_clean[['ship_clean', 'date_clean', 'source']].dropna(subset=['date_clean'])
            ], ignore_index=True)
            print(f"  {len(df_iws_clean.dropna(subset=['date_clean']))} eventos de limpeza IWS")

    # 2. Extrair datas de DOCAGEM como limpeza
    if not df_eventos.empty:
        df_docking = df_eventos[df_eventos['eventName'] == 'DOCAGEM'].copy()
        if not df_docking.empty:
            df_docking['date_clean'] = pd.to_datetime(df_docking['startGMTDate'], errors='coerce')
            df_docking['ship_clean'] = df_docking['shipName'].astype(str).str.lower().str.strip()
            df_docking['source'] = 'DOCAGEM'

            cleaning_events = pd.concat([
                cleaning_events,
                df_docking[['ship_clean', 'date_clean', 'source']].dropna(subset=['date_clean'])
            ], ignore_index=True)
            print(f"  {len(df_docking.dropna(subset=['date_clean']))} eventos de DOCAGEM")

    if cleaning_events.empty:
        print("Nenhum evento de limpeza encontrado")
        return df_events_ais

    # Remover duplicatas e ordenar
    cleaning_events = cleaning_events.sort_values('date_clean').drop_duplicates(
        subset=['ship_clean', 'date_clean'], keep='first'
    )

    print(f"  Total: {len(cleaning_events)} eventos de limpeza combinados")

    # 3. Calcular intervalo mediano por navio
    median_interval = cleaning_events.groupby('ship_clean')['date_clean'].apply(
        lambda g: g.sort_values().diff().dt.days.median()
    ).rename('median_interval').reset_index()
    median_interval['median_interval'].fillna(180, inplace=True)

    # 4. Calcular dias desde última limpeza (IWS ou DOCAGEM)
    def days_since_last_clean(row):
        s = str(row['shipName']).lower().strip()
        start = row['startGMTDate']
        if pd.isna(start):
            return np.nan, np.nan, 'unknown'

        cleans = cleaning_events[
            (cleaning_events['ship_clean'] == s) &
            (cleaning_events['date_clean'] <= start)
        ]

        if cleans.empty:
            return np.nan, np.nan, 'none'

        last_clean_idx = cleans['date_clean'].idxmax()
        last_clean = cleans.loc[last_clean_idx]
        days = (start - last_clean['date_clean']).days
        source = last_clean['source']

        median = median_interval[median_interval['ship_clean'] == s]['median_interval']
        median_val = median.values[0] if not median.empty else 180

        return days, median_val, source

    days_list = []
    median_list = []
    source_list = []

    for _, r in tqdm(df_events_ais.iterrows(), total=len(df_events_ais),
                     desc="Calculando dias desde limpeza"):
        d, med, src = days_since_last_clean(r)
        days_list.append(d)
        median_list.append(med)
        source_list.append(src)

    df_events_ais['days_since_clean'] = days_list
    df_events_ais['median_interval'] = median_list
    df_events_ais['clean_source'] = source_list

    # Estatísticas
    source_counts = pd.Series(source_list).value_counts()
    print(f"\n  Origem da última limpeza:")
    for src, count in source_counts.items():
        print(f"    {src}: {count} eventos")

    return df_events_ais

def create_fouling_percentage_target(df):
    """
    Cria target baseado em PORCENTAGEM de incrustação no casco (0-100%)

    A porcentagem representa a área do casco coberta por bioincrustação.
    Depois converte para escala IMO apenas para referência visual.

    Escala IMO MEPC.378(80) (apenas referência):
    0: Sem bioincrustação (0%)
    1: Microincrustação (biofilme/limo) (0-1%)
    2: Macroincrustação leve (1-15%)
    3: Macroincrustação moderada (16-40%)
    4: Macroincrustação pesada (41-100%)
    """
    df = df.copy()

    def estimate_fouling_percentage(row):
        """Estima porcentagem de área do casco com incrustação"""
        days = row.get('days_since_clean', np.nan)
        velocity_risk = row.get('velocity_risk', 2)
        idle_ratio = row.get('idle_time_ratio', 0)
        temp_risk = row.get('temp_risk', 0.5)
        region_risk = row.get('region_risk', 1.5)

        if pd.isna(days):
            return np.nan

        # Base de crescimento por tempo (em porcentagem)
        # Crescimento exponencial nos primeiros dias, depois linear
        if days < 14:
            base_pct = 0.5  # Biofilme inicial
        elif days < 42:
            base_pct = 2.0 + (days - 14) * 0.15  # Crescimento acelerado
        elif days < 90:
            base_pct = 6.2 + (days - 42) * 0.25  # Crescimento moderado
        elif days < 180:
            base_pct = 18.2 + (days - 90) * 0.22  # Crescimento contínuo
        elif days < 365:
            base_pct = 38.0 + (days - 180) * 0.15  # Crescimento desacelerando
        else:
            base_pct = 65.75 + (days - 365) * 0.08  # Crescimento lento

        # Modificadores baseados em condições operacionais
        velocity_modifier = velocity_risk * 3.5  # Alto impacto da velocidade
        idle_modifier = idle_ratio * 12.0  # Tempo parado acelera muito
        temp_modifier = temp_risk * 8.0  # Temperatura favorece crescimento
        region_modifier = (region_risk - 1.5) * 5.0  # Região biogeográfica

        final_pct = base_pct + velocity_modifier + idle_modifier + temp_modifier + region_modifier

        return np.clip(final_pct, 0, 100)

    df['fouling_percentage'] = df.apply(estimate_fouling_percentage, axis=1)

    # Converter porcentagem para escala IMO (apenas para referência)
    def percentage_to_imo_rating(pct):
        """Converte porcentagem para escala IMO"""
        if pd.isna(pct):
            return np.nan
        if pct < 0.5:
            return 0.0  # Sem incrustação
        elif pct < 1.0:
            return 0.5 + (pct - 0.5)  # Transição para micro
        elif pct < 15.0:
            return 1.0 + (pct - 1.0) / 14.0  # Microincrustação
        elif pct < 40.0:
            return 2.0 + (pct - 15.0) / 25.0  # Leve
        elif pct < 70.0:
            return 3.0 + (pct - 40.0) / 30.0  # Moderada
        else:
            return 4.0  # Pesada

    df['fouling_rating_imo'] = df['fouling_percentage'].apply(percentage_to_imo_rating)

    # Criar estágios baseados em porcentagem
    def get_fouling_stage_from_pct(pct):
        if pd.isna(pct):
            return np.nan
        if pct < 1.0:
            return 0  # Limpo/Micro
        elif pct < 15.0:
            return 1  # Leve
        elif pct < 40.0:
            return 2  # Moderado
        else:
            return 3  # Pesado

    df['fouling_stage'] = df['fouling_percentage'].apply(get_fouling_stage_from_pct)

    # Labels categóricos baseados em porcentagem
    def get_fouling_label_from_pct(pct):
        if pd.isna(pct):
            return np.nan
        if pct < 1.0:
            return 'clean'
        elif pct < 15.0:
            return 'light'
        elif pct < 40.0:
            return 'moderate'
        else:
            return 'heavy'

    df['fouling_label'] = df['fouling_percentage'].apply(get_fouling_label_from_pct)

    # Risk score combinado (mantido para compatibilidade)
    df['biofouling_risk_score'] = (
        0.4 * (df['days_since_clean'].fillna(90) / 180).clip(0, 1) +
        0.25 * (df['velocity_risk'] / 3) +
        0.2 * df['idle_time_ratio'] +
        0.15 * df['temp_risk']
    ).clip(0, 1)

    print("Target de Porcentagem de Fouling criado!")
    print(f"\nDistribuição de Porcentagem de Incrustação:")
    print(df['fouling_percentage'].describe())
    print(f"\nDistribuição de Rating IMO (referência):")
    print(df['fouling_rating_imo'].describe())

    return df

print("\n Processando IWS, DOCAGEM e criando target...")
if not df_events_ais.empty:
    df_events_ais = process_iws_and_docking_data(df_iws, df_eventos, df_events_ais)
    df_events_ais = create_fouling_percentage_target(df_events_ais)


 Processando IWS, DOCAGEM e criando target...
  28 eventos de limpeza IWS
  152 eventos de DOCAGEM
  Total: 180 eventos de limpeza combinados


Calculando dias desde limpeza: 100%|██████████| 8214/8214 [00:07<00:00, 1108.40it/s]



  Origem da última limpeza:
    DOCAGEM: 4738 eventos
    IWS: 2627 eventos
    none: 849 eventos
Target de Porcentagem de Fouling criado!

Distribuição de Porcentagem de Incrustação:
count    7365.000000
mean       71.844837
std        28.813374
min         0.000000
25%        50.544110
50%        80.301049
75%       100.000000
max       100.000000
Name: fouling_percentage, dtype: float64

Distribuição de Rating IMO (referência):
count    7365.000000
mean        3.582217
std         0.705288
min         0.000000
25%         3.351470
50%         4.000000
75%         4.000000
max         4.000000
Name: fouling_rating_imo, dtype: float64


## 6. MERGE COM CONSUMO E NAVIOS

In [14]:
print("\n🔗 Merging dados...")
if not df_events_ais.empty and 'sessionId' in df_consumo.columns:
    df_cons_sum = df_consumo.groupby('sessionId', as_index=False)['CONSUMED_QUANTITY'].sum()
    df_events_ais = df_events_ais.merge(df_cons_sum, on='sessionId', how='left')

if not df_events_ais.empty and not df_navios.empty:
    shipname_col = [c for c in df_navios.columns if 'nome' in c.lower() or 'name' in c.lower()]
    if shipname_col:
        snc = shipname_col[0]
        df_navios['ship_nav_low'] = df_navios[snc].astype(str).str.lower().str.strip()
        df_events_ais['ship_low'] = df_events_ais['shipName'].astype(str).str.lower().str.strip()
        df_events_ais = df_events_ais.merge(df_navios, left_on='ship_low', right_on='ship_nav_low', how='left')


🔗 Merging dados...


## 7. PREPARAR DATASET ML

In [15]:
features_v2 = [
    'speed_mean', 'speed_std', 'speed_min', 'speed_max',
    'duration_h', 'distance',
    'frac_stop', 'frac_low_speed', 'idle_days', 'low_speed_days',
    'velocity_risk', 'operation_continuity', 'speed_variability',
    'low_shear_exposure', 'biofouling_risk_score',
    'beaufort', 'seaCondition', 'lat_mean', 'lon_mean',
    'temp_proxy', 'temp_risk', 'region_risk',
    'days_since_clean', 'fouling_stage',
    'displacement'
]

if 'CONSUMED_QUANTITY' in df_events_ais.columns:
    features_v2.append('CONSUMED_QUANTITY')

features_available = [f for f in features_v2 if f in df_events_ais.columns]

print(f"\n📊 Features disponíveis: {len(features_available)}")

df_ml = df_events_ais.dropna(subset=['fouling_percentage'])[features_available + ['fouling_percentage', 'fouling_rating_imo', 'fouling_label', 'startGMTDate', 'shipName']].copy()
df_ml[features_available] = df_ml[features_available].fillna(0)

print(f"✅ Dataset ML: {df_ml.shape}")
print(f"   Target: Porcentagem de Incrustação (0-100%)")


📊 Features disponíveis: 26
✅ Dataset ML: (7365, 31)
   Target: Porcentagem de Incrustação (0-100%)


## 8.  VALIDAÇÃO TEMPORAL

In [16]:
df_ml_sorted = df_ml.sort_values('startGMTDate').reset_index(drop=True)

X = df_ml_sorted[features_available].values
y_reg = df_ml_sorted['fouling_percentage'].values  # Target principal: porcentagem
y_imo = df_ml_sorted['fouling_rating_imo'].values  # Referência IMO
y_clf = LabelEncoder().fit_transform(df_ml_sorted['fouling_label'].astype(str).values)

split_idx = int(len(df_ml_sorted) * 0.8)

X_train = X[:split_idx]
X_test = X[split_idx:]
y_train = y_reg[:split_idx]
y_test = y_reg[split_idx:]

print(f"✅ Treino: {X_train.shape[0]} | Teste: {X_test.shape[0]}")
print(f"   Target: Porcentagem de Incrustação (0-100%)")

✅ Treino: 5892 | Teste: 1473
   Target: Porcentagem de Incrustação (0-100%)


## 9. 🎯 MODELO ENSEMBLE

In [17]:
print("\n Treinando ensemble...")

models = {
    'XGBoost': xgb.XGBRegressor(
        n_estimators=300, learning_rate=0.03, max_depth=6,
        subsample=0.8, colsample_bytree=0.8, random_state=42, verbosity=0
    ),
    'LightGBM': lgb.LGBMRegressor(
        n_estimators=300, learning_rate=0.03, max_depth=6,
        random_state=42, verbosity=-1
    ),
    'RandomForest': RandomForestRegressor(
        n_estimators=200, max_depth=10, random_state=42, n_jobs=-1
    ),
    'GradientBoosting': GradientBoostingRegressor(
        n_estimators=200, learning_rate=0.05, max_depth=5, random_state=42
    )
}

predictions = {}
model_scores = {}

for name, model in models.items():
    print(f"\nTreinando {name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    predictions[name] = y_pred

    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    model_scores[name] = {'MAE': mae, 'RMSE': rmse, 'R2': r2}
    print(f"  MAE: {mae:.4f} | RMSE: {rmse:.4f} | R²: {r2:.4f}")

# Ensemble com pesos
maes = [model_scores[name]['MAE'] for name in models.keys()]
weights = [1/mae for mae in maes]
weights = [w/sum(weights) for w in weights]

y_pred_ensemble = sum(predictions[name] * weight for name, weight in zip(models.keys(), weights))

mae_ensemble = mean_absolute_error(y_test, y_pred_ensemble)
rmse_ensemble = np.sqrt(mean_squared_error(y_test, y_pred_ensemble))
r2_ensemble = r2_score(y_test, y_pred_ensemble)

print(f"\n{'='*60}")
print(" ENSEMBLE:")
print(f"  MAE: {mae_ensemble:.4f}")
print(f"  RMSE: {rmse_ensemble:.4f}")
print(f"  R²: {r2_ensemble:.4f}")
print(f"{'='*60}")


 Treinando ensemble...

Treinando XGBoost...
  MAE: 0.4822 | RMSE: 0.7648 | R²: 0.9995

Treinando LightGBM...
  MAE: 0.4512 | RMSE: 0.7206 | R²: 0.9995

Treinando RandomForest...
  MAE: 0.5541 | RMSE: 0.9892 | R²: 0.9992

Treinando GradientBoosting...
  MAE: 0.5325 | RMSE: 0.8056 | R²: 0.9994

 ENSEMBLE:
  MAE: 0.4400
  RMSE: 0.7133
  R²: 0.9996


## 10.  IMPACTO ECONÔMICO


In [18]:
def compute_fuel_penalty_from_fouling(fouling_percentage, baseline_consumption):
    """
    Calcula penalidade de combustível baseado na PORCENTAGEM de incrustação

    Baseado em estudos científicos:
    - 0-1%: Biofilme inicial, penalidade mínima (0-5%)
    - 1-15%: Macroincrustação leve (5-10%)
    - 15-40%: Macroincrustação moderada (10-18%)
    - 40-70%: Macroincrustação pesada (18-25%)
    - 70-100%: Macroincrustação crítica (25-35%)
    """
    PRICE_PER_TON = 650
    CO2_PER_TON = 3.114

    # Calcular penalidade baseada em porcentagem de incrustação
    if fouling_percentage < 0.5:
        penalty = 0.0
    elif fouling_percentage < 1.0:
        # Biofilme inicial: 0-5%
        penalty = 0.05 * (fouling_percentage / 1.0)
    elif fouling_percentage < 15.0:
        # Leve: 5-10%
        penalty = 0.05 + 0.05 * ((fouling_percentage - 1.0) / 14.0)
    elif fouling_percentage < 40.0:
        # Moderada: 10-18%
        penalty = 0.10 + 0.08 * ((fouling_percentage - 15.0) / 25.0)
    elif fouling_percentage < 70.0:
        # Pesada: 18-25%
        penalty = 0.18 + 0.07 * ((fouling_percentage - 40.0) / 30.0)
    else:
        # Crítica: 25-35%
        penalty = 0.25 + 0.10 * min((fouling_percentage - 70.0) / 30.0, 1.0)

    extra_fuel = baseline_consumption * penalty

    return {
        'fouling_percentage': fouling_percentage,
        'fuel_penalty_pct': penalty * 100,
        'extra_fuel_tons_day': extra_fuel,
        'extra_cost_usd_day': extra_fuel * PRICE_PER_TON,
        'extra_cost_usd_month': extra_fuel * PRICE_PER_TON * 30,
        'extra_cost_usd_year': extra_fuel * PRICE_PER_TON * 365,
        'extra_co2_tons_year': extra_fuel * CO2_PER_TON * 365
    }

print("\n Calculando impacto econômico...")
baseline = 40
all_impacts = [compute_fuel_penalty_from_fouling(pred, baseline) for pred in y_pred_ensemble]
df_impacts = pd.DataFrame(all_impacts)

print(f"Custo Extra Médio/Dia: ${df_impacts['extra_cost_usd_day'].mean():,.2f}")
print(f"Custo Extra Médio/Mês: ${df_impacts['extra_cost_usd_month'].mean():,.2f}")
print(f"Custo Extra Médio/Ano: ${df_impacts['extra_cost_usd_year'].mean():,.2f}")
print(f"CO2 Extra Médio/Ano: {df_impacts['extra_co2_tons_year'].mean():,.2f} tons")


 Calculando impacto econômico...
Custo Extra Médio/Dia: $6,607.33
Custo Extra Médio/Mês: $198,219.84
Custo Extra Médio/Ano: $2,411,674.76
CO2 Extra Médio/Ano: 11,553.78 tons


## 11.  ANÁLISE DE CENÁRIOS

In [19]:
def simulate_cleaning_scenarios(current_fouling_pct, days_since_clean, baseline=40):
    """
    Simula cenários de limpeza baseado em porcentagem de incrustação

    Args:
        current_fouling_pct: Porcentagem atual de incrustação (0-100%)
        days_since_clean: Dias desde última limpeza
        baseline: Consumo base em tons/dia
    """
    CLEANING_COST = 50000
    DOWNTIME_COST = 24 * 5000
    DAYS_AHEAD = 180

    scenarios = {}

    # Cenário 1: Não fazer nada
    # Crescimento estimado: ~0.15% por dia em condições normais
    growth_rate = 0.15
    future_fouling_pct = min(current_fouling_pct + (DAYS_AHEAD * growth_rate), 100.0)

    current_impact = compute_fuel_penalty_from_fouling(current_fouling_pct, baseline)
    future_impact = compute_fuel_penalty_from_fouling(future_fouling_pct, baseline)
    avg_cost = (current_impact['extra_cost_usd_day'] + future_impact['extra_cost_usd_day']) / 2

    scenarios['Não Fazer Limpeza'] = {
        'total_cost': avg_cost * DAYS_AHEAD,
        'final_fouling_pct': future_fouling_pct,
        'final_fouling_desc': f"{future_fouling_pct:.1f}%"
    }

    # Cenário 2: Limpar agora
    post_clean_pct = 0.5  # Após limpeza: ~0.5%
    future_clean_pct = min(post_clean_pct + (DAYS_AHEAD * growth_rate * 0.8), 30.0)  # Crescimento mais lento

    post_impact = compute_fuel_penalty_from_fouling(post_clean_pct, baseline)
    future_impact_clean = compute_fuel_penalty_from_fouling(future_clean_pct, baseline)
    avg_cost_clean = (post_impact['extra_cost_usd_day'] + future_impact_clean['extra_cost_usd_day']) / 2

    scenarios['Fazer Limpeza'] = {
        'total_cost': CLEANING_COST + DOWNTIME_COST + (avg_cost_clean * DAYS_AHEAD),
        'final_fouling_pct': future_clean_pct,
        'final_fouling_desc': f"{future_clean_pct:.1f}%"
    }

    return scenarios

print("\n Simulando cenários...")
example_fouling_pct = y_pred_ensemble[0]
example_days = df_ml_sorted.iloc[split_idx]['days_since_clean']

scenarios = simulate_cleaning_scenarios(example_fouling_pct, example_days)

print(f"\nIncrustação atual: {example_fouling_pct:.1f}%")
for name, data in scenarios.items():
    print(f"\n{name}:")
    print(f"  Custo total: ${data['total_cost']:,.2f}")
    print(f"  Incrustação final: {data['final_fouling_desc']}")

best = min(scenarios.items(), key=lambda x: x[1]['total_cost'])[0]


 Simulando cenários...

Incrustação atual: 64.4%

Não Fazer Limpeza:
  Custo total: $1,305,865.93
  Incrustação final: 91.4%

Fazer Limpeza:
  Custo total: $515,664.80
  Incrustação final: 22.1%


## 12. SALVAR MODELOS

In [20]:
print("\n Salvando modelos...")
for name, model in models.items():
    filename = f"model_{name.lower().replace(' ', '_')}_v2.pkl"
    with open(filename, 'wb') as f:
        pickle.dump(model, f)

metadata = {
    'features': features_available,
    'weights': dict(zip(models.keys(), weights)),
    'mae': mae_ensemble,
    'rmse': rmse_ensemble,
    'r2': r2_ensemble
}

with open('model_metadata_v2.pkl', 'wb') as f:
    pickle.dump(metadata, f)

print(" Modelos salvos")

# 13. RESUMO


print("\n" + "="*80)
print(" RESUMO SOLUÇÃO DE PREDIÇÃO")
print("="*80)

print(f"\n PERFORMANCE:")
print(f"  MAE:  {mae_ensemble:.4f}")
print(f"  RMSE: {rmse_ensemble:.4f}")
print(f"  R²:   {r2_ensemble:.4f}")


 Salvando modelos...
 Modelos salvos

 RESUMO SOLUÇÃO DE PREDIÇÃO

 PERFORMANCE:
  MAE:  0.4400
  RMSE: 0.7133
  R²:   0.9996


## 13. FOULING RATING POR NAVIO (FROTA)

In [21]:
print_header("ANÁLISE DA FROTA")

# Pegar último evento de cada navio
df_ml_sorted_final = df_ml_sorted.copy()

# Verificar se shipName existe, senão usar índice
if 'shipName' not in df_ml_sorted_final.columns:
    # Adicionar shipName do df_ml original
    df_ml_sorted_final = df_ml_sorted_final.merge(
        df_ml[['startGMTDate', 'shipName']],
        on='startGMTDate',
        how='left'
    )

df_ml_sorted_final['shipName_clean'] = df_ml_sorted_final['shipName'].astype(str).str.strip()

# Último evento de cada navio (mais recente)
ultimos_eventos = df_ml_sorted_final.groupby('shipName_clean').last().reset_index()

# Calcular impacto econômico para cada navio
resultados_frota = []

for _, navio_data in ultimos_eventos.iterrows():
    ship_name = navio_data['shipName_clean']
    fouling_pct = navio_data['fouling_percentage']
    fouling_imo = navio_data.get('fouling_rating_imo', np.nan)
    days_clean = navio_data.get('days_since_clean', np.nan)

    # Calcular impacto
    baseline = 40  # tons/dia (ajustar se tiver dados específicos)
    impacto = compute_fuel_penalty_from_fouling(fouling_pct, baseline)

    # Classificar por porcentagem
    if fouling_pct < 1.0:
        classificacao = "Limpo"
        acao = "OK"
    elif fouling_pct < 15.0:
        classificacao = "Leve"
        acao = "Monitorar"
    elif fouling_pct < 40.0:
        classificacao = "Moderada"
        acao = "Recomendada"
    elif fouling_pct < 70.0:
        classificacao = "Pesada"
        acao = "Urgente"
    else:
        classificacao = "Crítica"
        acao = "Imediata"

    resultados_frota.append({
        'Navio': ship_name,
        'Incrust.%': round(fouling_pct, 1),
        'IMO': round(fouling_imo, 2) if not pd.isna(fouling_imo) else 'N/A',
        'Status': classificacao,
        'Dias': int(days_clean) if not pd.isna(days_clean) else 'N/A',
        'Penalidade': f"{impacto['fuel_penalty_pct']:.1f}%",
        'Custo/Ano': f"${impacto['extra_cost_usd_year']/1e6:.2f}M",
        'CO2/Ano': f"{impacto['extra_co2_tons_year']/1e3:.1f}kt",
        'Ação': acao
    })

df_frota = pd.DataFrame(resultados_frota)

# Ordenar por Porcentagem de Incrustação (maior primeiro)
df_frota = df_frota.sort_values('Incrust.%', ascending=False)

# Imprimir tabela
if RICH_AVAILABLE:
    table = Table(title="Condição da Frota", box=box.SIMPLE_HEAD, show_header=True, header_style="bold cyan")

    for col in df_frota.columns:
        justify = "right" if col in ['Incrust.%', 'IMO', 'Dias', 'Penalidade'] else "left"
        table.add_column(col, justify=justify)

    for _, row in df_frota.iterrows():
        # Colorir baseado na classificação
        status = row['Status']
        if status == 'Crítica':
            style = "bold red"
        elif status == 'Pesada':
            style = "red"
        elif status == 'Moderada':
            style = "yellow"
        elif status == 'Leve':
            style = "blue"
        else:
            style = "green"

        table.add_row(*[str(val) for val in row], style=style)

    console.print(table)
else:
    print("\n" + df_frota.to_string(index=False))

# Estatísticas da frota
print_header("ESTATÍSTICAS DA FROTA")

fouling_pct_values = df_frota['Incrust.%'].values

# Distribuição por categoria (baseada em porcentagem)
clean_count = (fouling_pct_values < 1.0).sum()
leve_count = ((fouling_pct_values >= 1.0) & (fouling_pct_values < 15.0)).sum()
moderada_count = ((fouling_pct_values >= 15.0) & (fouling_pct_values < 40.0)).sum()
pesada_count = ((fouling_pct_values >= 40.0) & (fouling_pct_values < 70.0)).sum()
critica_count = (fouling_pct_values >= 70.0).sum()

total_navios = len(fouling_pct_values)

print_info(f"\nDistribuição:")
print_info(f"  Limpo (0-1%):     {clean_count:2d} navios ({clean_count/total_navios*100:5.1f}%)")
print_info(f"  Leve (1-15%):     {leve_count:2d} navios ({leve_count/total_navios*100:5.1f}%)")
print_info(f"  Moderada (15-40%): {moderada_count:2d} navios ({moderada_count/total_navios*100:5.1f}%)")
print_info(f"  Pesada (40-70%):   {pesada_count:2d} navios ({pesada_count/total_navios*100:5.1f}%)")
print_info(f"  Crítica (70-100%): {critica_count:2d} navios ({critica_count/total_navios*100:5.1f}%)")

# Priorização de ações
urgente_count = (fouling_pct_values >= 40.0).sum()
recomendada_count = ((fouling_pct_values >= 15.0) & (fouling_pct_values < 40.0)).sum()
monitorar_count = ((fouling_pct_values >= 1.0) & (fouling_pct_values < 15.0)).sum()
critica_count_frota = critica_count  # Salvar para comparação na validação

print_info(f"\nAções Requeridas:")
if urgente_count > 0:
    print_warning(f"{urgente_count} navios requerem limpeza urgente (≥40%)")
if recomendada_count > 0:
    print_info(f"  {recomendada_count} navios requerem limpeza recomendada (15-40%)")
if monitorar_count > 0:
    print_info(f"  {monitorar_count} navios requerem monitoramento (1-15%)")

# Impacto econômico total
custo_ano_values = df_frota['Custo/Ano'].str.replace('$', '').str.replace('M', '').astype(float) * 1e6
co2_ano_values = df_frota['CO2/Ano'].str.replace('kt', '').astype(float) * 1e3

total_custo = custo_ano_values.sum()
total_co2 = co2_ano_values.sum()

print_info(f"\nImpacto Econômico Total:")
print_info(f"  Custo Extra/Ano: ${total_custo/1e6:.2f}M | Médio/Navio: ${total_custo/total_navios/1e6:.2f}M")
print_info(f"  CO2 Extra/Ano: {total_co2/1e3:.1f}kt | Médio/Navio: {total_co2/total_navios/1e3:.1f}kt")

# Salvar resultados detalhados
df_frota.to_csv('fouling_por_navio.csv', index=False)
print_success("Resultados salvos em: fouling_por_navio.csv")

ANÁLISE DA FROTA

────────────────

                                             Condição da Frota                                             
                                                                                                           
  Navio               Incrust.%    IMO   Status     Dias   Penalidade   Custo/Ano   CO2/Ano   Ação         
 ───────────────────────────────────────────────────────────────────────────────────────────────────────── 
  BRUNO LIMA              100.0    4.0   Crítica     565        35.0%   $3.32M      15.9kt    Imediata     
  DANIEL PEREIRA          100.0    4.0   Crítica     771        35.0%   $3.32M      15.9kt    Imediata     
  VICTOR OLIVEIRA         100.0    4.0   Crítica     915        35.0%   $3.32M      15.9kt    Imediata     
  MARCOS CAVALCANTI       100.0    4.0   Crítica     666        35.0%   $3.32M      15.9kt    Imediata     
  GABRIELA MARTINS         81.9    4.0   Crítica     598        29.0%   $2.75M      13.2kt    Imediata     
  MARIA VALENTINA          73.5    4.0   Crítica     374        26.2%   $2.48M      11.9kt    Imediata     
  FELIPE RIBEIRO           72.3    4.0   Crítica     478        25.8%   $2.44M      11.7kt    Imediata     
  RODRIGO PINHEIRO         70.8    4.0   Crítica     460        25.3%   $2.40M      11.5kt    Imediata     
  EDUARDO COSTA            50.1   3.34   Pesada      181        20.4%   $1.93M      9.3kt     Urgente      
  GISELLE CARVALHO         45.5   3.19   Pesada      247        19.3%   $1.83M      8.8kt     Urgente      
  THIAGO FERNANDES         34.7   2.79   Moderada     42        16.3%   $1.55M      7.4kt     Recomendada  
  RAUL MARTINS             32.0   2.68   Moderada    148        15.4%   $1.46M      7.0kt     Recomendada  
  RICARDO BARBOSA          29.0   2.56   Moderada      6        14.5%   $1.37M      6.6kt     Recomendada  
  CARLA SILVA              26.1   2.44   Moderada     41        13.5%   $1.28M      6.2kt     Recomendada  
  PAULO MOURA              22.5    2.3   Moderada    105        12.4%   $1.18M      5.6kt     Recomendada  
  RAFAEL SANTOS            10.7   1.69   Leve         42         8.5%   $0.80M      3.8kt     Monitorar    
  HENRIQUE ALVES            9.9   1.64   Leve         53         8.2%   $0.78M      3.7kt     Monitorar    
  ROMARIO SILVA             6.7   1.41   Leve          9         7.0%   $0.67M      3.2kt     Monitorar   

ESTATÍSTICAS DA FROTA

─────────────────────

...

3 navios requerem monitoramento (1-15%)

Impacto Econômico Total:

Custo Extra/Ano: $36.20M | Médio/Navio: $2.01M

CO2 Extra/Ano: 173.5kt | Médio/Navio: 9.6kt

✓ Resultados salvos em: fouling_por_navio.csv

## 14. VALIDAÇÃO COM NAVIOS DE TESTE

In [22]:
print_header("VALIDAÇÃO COM NAVIOS DE TESTE")

# Verificar se existem dados de validação
VALIDATION_PATH = "Hackathon Transpetro/Mais Dados/"


if os.path.exists(VALIDATION_PATH):

    try:
        # Carregar dados de validação com fallback para arquivos principais
        print_info("Carregando dados de validação...")

        # Tentar carregar eventos de validação, senão usar principal
        try:
            df_eventos_val = pd.read_csv(f"{VALIDATION_PATH}Eventos_Validacao 1.CSV")
        except FileNotFoundError:
            df_eventos_val = df_eventos.copy()

        # Tentar carregar consumo de validação, senão usar principal
        try:
            df_consumo_val = pd.read_csv(f"{VALIDATION_PATH}Consumo_Validacao 1.CSV")
            print_success(f"  Consumo de validação: {df_consumo_val.shape[0]} registros")
        except FileNotFoundError:
            print_warning("  Consumo_Validacao não encontrado, usando ResultadoQueryConsumo.csv")
            df_consumo_val = df_consumo.copy()

        # Tentar carregar dados navios de validação, senão usar principal
        try:
            df_navios_val = pd.read_excel(f"{VALIDATION_PATH}Dados navios Validação 1.xlsx")
            print_success(f"  Dados navios de validação: {df_navios_val.shape[0]} registros")
        except FileNotFoundError:
            print_warning("  Dados navios Validação não encontrado, usando Dados navios Hackathon.xlsx")
            df_navios_val = df_navios.copy()

        # Carregar AIS dos navios teste com fallback
        df_ais_val = pd.DataFrame()

        # Tentar carregar AIS TESTE 2
        try:
            df_ais_teste2 = pd.read_csv(f"{VALIDATION_PATH}AIS_NAVIO TESTE 2 1.csv")
            df_ais_val = pd.concat([df_ais_val, df_ais_teste2], ignore_index=True)
            print_success(f"  AIS TESTE 2: {df_ais_teste2.shape[0]} registros")
        except FileNotFoundError:
            print_warning("  AIS_NAVIO TESTE 2 não encontrado")

        # Tentar carregar AIS TESTE 3
        try:
            df_ais_teste3 = pd.read_csv(f"{VALIDATION_PATH}AIS_NAVIO TESTE 3 1.csv")
            df_ais_val = pd.concat([df_ais_val, df_ais_teste3], ignore_index=True)
            print_success(f"  AIS TESTE 3: {df_ais_teste3.shape[0]} registros")
        except FileNotFoundError:
            print_warning("  AIS_NAVIO TESTE 3 não encontrado")



        # Template de resultado
        try:
            df_resultado = pd.read_excel(f"{VALIDATION_PATH}RESULTADO Validação 1.xlsx", header=0)

            # Renomear colunas corretamente
            if 'Unnamed: 0' in df_resultado.columns:
                df_resultado.columns = ['Embarcação', 'Data', 'Condição do Casco']

            # Remover primeira linha se for cabeçalho duplicado
            if len(df_resultado) > 0 and str(df_resultado.iloc[0]['Embarcação']).strip().upper() == 'EMBARCAÇÃO':
                df_resultado = df_resultado.iloc[1:].reset_index(drop=True)

            # Remover linhas vazias e inválidas
            df_resultado = df_resultado.dropna(subset=['Embarcação', 'Data'], how='any')
            df_resultado = df_resultado[df_resultado['Embarcação'].str.strip() != '']
            df_resultado = df_resultado[~df_resultado['Embarcação'].str.contains('teste 1', case=False, na=False)]


        except FileNotFoundError:
            print_warning("  RESULTADO Validação não encontrado, criando template com todos os navios")
            # Criar template com todos os navios únicos dos eventos
            navios_unicos = df_eventos_val['shipName'].unique()
            datas_recentes = df_eventos_val.groupby('shipName')['startGMTDate'].max().reset_index()
            df_resultado = pd.DataFrame({
                'Embarcação': datas_recentes['shipName'],
                'Data': datas_recentes['startGMTDate'],
                'Condição do Casco': ''
            })

        print_success(f"\nResumo dos dados de validação:")
        print_info(f"  Eventos: {df_eventos_val.shape[0]} | AIS: {df_ais_val.shape[0]} | Template: {df_resultado.shape[0]}")

        # Pré-processar dados de validação
        df_eventos_val.columns = df_eventos_val.columns.str.strip()
        df_consumo_val.columns = df_consumo_val.columns.str.strip()
        df_ais_val.columns = df_ais_val.columns.str.strip()

        # Parse datetimes
        for c in ["startGMTDate", "endGMTDate"]:
            if c in df_eventos_val.columns:
                df_eventos_val[c] = pd.to_datetime(df_eventos_val[c], format='%d/%m/%Y %H:%M', errors='coerce')

        # Renomear SESSION_ID
        if "SESSION_ID" in df_consumo_val.columns:
            df_consumo_val.rename(columns={"SESSION_ID": "sessionId"}, inplace=True)

        # Processar AIS
        if 'DATAHORA' in df_ais_val.columns:
            df_ais_val['DATETIME'] = pd.to_datetime(df_ais_val['DATAHORA'], errors='coerce')

        if 'VELOCIDADE' in df_ais_val.columns:
            df_ais_val['speed_kn'] = pd.to_numeric(df_ais_val['VELOCIDADE'], errors='coerce')

        if 'NOME' in df_ais_val.columns:
            df_ais_val['shipName_ais'] = df_ais_val['NOME'].astype(str)

        # Identificar quais navios do template precisamos processar
        navios_no_template = df_resultado['Embarcação'].str.upper().unique()

        # Agregar AIS por evento - COMBINAR eventos de validação E principais
        print_info("Processando eventos de validação...")
        df_events_ais_val_only = aggregate_ais_by_event(df_eventos_val, df_ais_val)

        # Verificar quais navios do template NÃO estão nos eventos de validação
        if not df_events_ais_val_only.empty:
            navios_em_validacao = set(df_events_ais_val_only['shipName'].str.upper().unique())
        else:
            navios_em_validacao = set()

        navios_faltantes = set(navios_no_template) - navios_em_validacao

        if navios_faltantes:

            # Filtrar eventos principais para incluir APENAS os navios que faltam
            df_eventos_faltantes = df_eventos[df_eventos['shipName'].str.upper().isin(navios_faltantes)].copy()

            if not df_eventos_faltantes.empty:
                df_events_ais_main_filtered = aggregate_ais_by_event(df_eventos_faltantes, df_ais)
            else:
                df_events_ais_main_filtered = pd.DataFrame()
        else:
            df_events_ais_main_filtered = pd.DataFrame()

        # Combinar eventos de validação + principais (apenas navios faltantes)
        if not df_events_ais_val_only.empty and not df_events_ais_main_filtered.empty:
            df_events_ais_val = pd.concat([df_events_ais_val_only, df_events_ais_main_filtered], ignore_index=True)
        elif not df_events_ais_val_only.empty:
            df_events_ais_val = df_events_ais_val_only
        elif not df_events_ais_main_filtered.empty:
            df_events_ais_val = df_events_ais_main_filtered
        else:
            df_events_ais_val = pd.DataFrame()


        if not df_events_ais_val.empty:
            # Criar features avançadas
            df_events_ais_val = create_advanced_features(df_events_ais_val)

            # Processar IWS e DOCAGEM para calcular dias desde limpeza CORRETAMENTE
            # Combinar eventos de validação + principais para buscar todas as docagens
            print_info("Calculando dias desde última limpeza (IWS + DOCAGEM)...")

            # Combinar eventos de validação e principais para ter todas as docagens
            df_eventos_combinados = pd.concat([df_eventos_val, df_eventos], ignore_index=True)

            # Processar com eventos combinados
            df_events_ais_val = process_iws_and_docking_data(df_iws, df_eventos_combinados, df_events_ais_val)

            # Criar estágio de fouling baseado em days_since_clean
            def get_stage(days):
                if pd.isna(days):
                    return 2
                if days < 14:
                    return 0
                elif days < 42:
                    return 1
                elif days < 90:
                    return 2
                else:
                    return 3

            # Garantir que fouling_stage existe
            if 'fouling_stage' not in df_events_ais_val.columns:
                df_events_ais_val['fouling_stage'] = df_events_ais_val['days_since_clean'].apply(get_stage)

            # Criar biofouling risk score
            df_events_ais_val['biofouling_risk_score'] = (
                0.4 * (df_events_ais_val['days_since_clean'].fillna(90) / 180).clip(0, 1) +
                0.25 * (df_events_ais_val['velocity_risk'] / 3) +
                0.2 * df_events_ais_val['idle_time_ratio'] +
                0.15 * df_events_ais_val['temp_risk']
            ).clip(0, 1)

            # Merge com consumo
            if 'sessionId' in df_consumo_val.columns:
                df_cons_sum = df_consumo_val.groupby('sessionId', as_index=False)['CONSUMED_QUANTITY'].sum()
                df_events_ais_val = df_events_ais_val.merge(df_cons_sum, on='sessionId', how='left')

            # Preparar features para predição
            features_missing = [f for f in features_available if f not in df_events_ais_val.columns]
            for feat in features_missing:
                df_events_ais_val[feat] = 0

            df_pred_val = df_events_ais_val[features_available + ['shipName', 'startGMTDate']].copy()
            df_pred_val[features_available] = df_pred_val[features_available].fillna(0)

            X_val = df_pred_val[features_available].values


            # Fazer predições com ensemble
            predictions_val = {}
            for name, model in models.items():
                pred = model.predict(X_val)
                predictions_val[name] = pred

            # Ensemble com pesos
            y_pred_val_ensemble = sum(predictions_val[name] * weight for name, weight in zip(models.keys(), weights))

            df_pred_val['fouling_percentage'] = y_pred_val_ensemble

            # Converter para IMO
            def percentage_to_imo_rating(pct):
                if pd.isna(pct):
                    return np.nan
                if pct < 0.5:
                    return 0.0
                elif pct < 1.0:
                    return 0.5 + (pct - 0.5)
                elif pct < 15.0:
                    return 1.0 + (pct - 1.0) / 14.0
                elif pct < 40.0:
                    return 2.0 + (pct - 15.0) / 25.0
                elif pct < 70.0:
                    return 3.0 + (pct - 40.0) / 30.0
                else:
                    return 4.0

            df_pred_val['fouling_rating_imo'] = df_pred_val['fouling_percentage'].apply(percentage_to_imo_rating)




            # Converter datas (podem já estar como datetime ou string)
            if 'Data' in df_resultado.columns:
                df_resultado['Data'] = pd.to_datetime(df_resultado['Data'], errors='coerce')


            resultados_preenchidos = []
            navios_sem_dados = []

            for idx, row in df_resultado.iterrows():
                navio = str(row.get('Embarcação', row.get('Navio', ''))).strip()
                data_alvo = row.get('Data')

                # Verificar se temos dados válidos
                if pd.isna(data_alvo) or navio == '' or navio == 'nan':
                    navios_sem_dados.append(navio)
                    continue

                # Buscar predições para este navio (busca mais robusta)
                # Tentar match exato primeiro
                mask = df_pred_val['shipName'].str.upper() == navio.upper()
                if not mask.any():
                    # Tentar match parcial
                    mask = df_pred_val['shipName'].str.upper().str.contains(navio.upper(), na=False)

                preds_navio = df_pred_val[mask].copy()

                if preds_navio.empty:
                    # Não incluir navios sem predições
                    navios_sem_dados.append(navio)
                    continue

                # Encontrar predições antes e depois da data alvo
                preds_antes = preds_navio[preds_navio['startGMTDate'] <= data_alvo]
                preds_depois = preds_navio[preds_navio['startGMTDate'] > data_alvo]

                if not preds_antes.empty and not preds_depois.empty:
                    # Interpolar entre antes e depois
                    pred_antes = preds_antes.iloc[-1]  # Última antes
                    pred_depois = preds_depois.iloc[0]  # Primeira depois

                    # Interpolação linear
                    dias_antes = (data_alvo - pred_antes['startGMTDate']).days
                    dias_depois = (pred_depois['startGMTDate'] - data_alvo).days
                    total_dias = dias_antes + dias_depois

                    if total_dias > 0:
                        peso_antes = dias_depois / total_dias
                        peso_depois = dias_antes / total_dias
                        pct = pred_antes['fouling_percentage'] * peso_antes + pred_depois['fouling_percentage'] * peso_depois
                        imo = pred_antes['fouling_rating_imo'] * peso_antes + pred_depois['fouling_rating_imo'] * peso_depois
                    else:
                        pct = pred_antes['fouling_percentage']
                        imo = pred_antes['fouling_rating_imo']

                elif not preds_antes.empty:
                    # Usar última predição antes da data
                    closest = preds_antes.iloc[-1]
                    pct = closest['fouling_percentage']
                    imo = closest['fouling_rating_imo']

                elif not preds_depois.empty:
                    # Usar primeira predição depois da data
                    closest = preds_depois.iloc[0]
                    pct = closest['fouling_percentage']
                    imo = closest['fouling_rating_imo']

                else:
                    # Fallback: predição mais próxima
                    preds_navio['time_diff'] = (preds_navio['startGMTDate'] - data_alvo).abs()
                    closest = preds_navio.loc[preds_navio['time_diff'].idxmin()]
                    pct = closest['fouling_percentage']
                    imo = closest['fouling_rating_imo']

                # Classificar
                if pct < 1.0:
                    classe = "Limpo/Micro"
                elif pct < 15.0:
                    classe = "Leve"
                elif pct < 40.0:
                    classe = "Moderada"
                elif pct < 70.0:
                    classe = "Pesada"
                else:
                    classe = "Crítica"

                condicao = f"{pct:.1f}% ({classe}, IMO {imo:.2f})"

                # Formatar data corretamente para Excel
                data_formatada = data_alvo.strftime('%d/%m/%Y') if pd.notna(data_alvo) else ''

                resultados_preenchidos.append({
                    'Embarcação': navio,
                    'Data': data_formatada,
                    'Condição do Casco': condicao
                })

            # Informar sobre navios excluídos (apenas se houver)
            navios_validos = [n for n in navios_sem_dados if n and n != 'nan' and 'teste 1' not in n.lower()]
            df_resultado_final = pd.DataFrame(resultados_preenchidos)

            # Salvar resultado
            output_file = 'RESULTADO_Validacao_Preenchido.xlsx'
            df_resultado_final.to_excel(output_file, index=False)

            print_success(f"Resultado salvo em: {output_file}")

            # Relatório detalhado por navio (similar à frota principal)
            print_header("RELATÓRIO DE VALIDAÇÃO")


            # Criar relatório baseado nas DATAS DE AVALIAÇÃO do template
            # Cada linha do template = uma predição separada
            resultados_validacao = []

            # Usar os resultados já preenchidos (que têm as predições para cada data)
            for resultado in resultados_preenchidos:
                navio = resultado['Embarcação']
                data_str = resultado['Data']
                condicao = resultado['Condição do Casco']

                # Extrair porcentagem e IMO da string de condição
                # Formato: "71.6% (Crítica, IMO 4.00)"
                if '%' in condicao and 'IMO' in condicao:
                    try:
                        pct_str = condicao.split('%')[0]
                        fouling_pct = float(pct_str)

                        imo_str = condicao.split('IMO ')[1].split(')')[0]
                        fouling_imo = float(imo_str)

                        classe = condicao.split('(')[1].split(',')[0]

                        # Calcular impacto
                        baseline = 40
                        impacto = compute_fuel_penalty_from_fouling(fouling_pct, baseline)

                        # Determinar ação
                        if fouling_pct < 1.0:
                            acao = "OK"
                        elif fouling_pct < 15.0:
                            acao = "Monitorar"
                        elif fouling_pct < 40.0:
                            acao = "Recomendada"
                        elif fouling_pct < 70.0:
                            acao = "Urgente"
                        else:
                            acao = "Imediata"

                        # Buscar dias desde limpeza para esta data específica
                        # Converter data string de volta para datetime para buscar
                        data_dt = pd.to_datetime(data_str, format='%d/%m/%Y')

                        # Buscar predições próximas a esta data
                        mask = df_pred_val['shipName'].str.upper() == navio.upper()
                        preds_navio = df_pred_val[mask]

                        if not preds_navio.empty:
                            # Encontrar predição mais próxima desta data
                            preds_navio_copy = preds_navio.copy()
                            preds_navio_copy['time_diff'] = (preds_navio_copy['startGMTDate'] - data_dt).abs()
                            closest_pred = preds_navio_copy.loc[preds_navio_copy['time_diff'].idxmin()]
                            days_clean = closest_pred.get('days_since_clean', np.nan)
                        else:
                            days_clean = np.nan

                        resultados_validacao.append({
                            'Navio': navio,
                            'Data': data_str,
                            'Incrust.%': round(fouling_pct, 1),
                            'IMO': round(fouling_imo, 2),
                            'Status': classe,
                            'Dias*': f"{int(days_clean)}*" if not pd.isna(days_clean) else 'N/A',
                            'Penalidade': f"{impacto['fuel_penalty_pct']:.1f}%",
                            'Custo/Ano': f"${impacto['extra_cost_usd_year']/1e6:.2f}M",
                            'CO2/Ano': f"{impacto['extra_co2_tons_year']/1e3:.1f}kt",
                            'Ação': acao
                        })
                    except Exception as e:
                        print_warning(f"Erro ao processar resultado para {navio}: {e}")

            df_validacao = pd.DataFrame(resultados_validacao)
            df_validacao = df_validacao.sort_values('Incrust.%', ascending=False)

            # Imprimir tabela
            if RICH_AVAILABLE:
                table = Table(title="Navios de Validação", box=box.SIMPLE_HEAD, show_header=True, header_style="bold cyan")

                for col in df_validacao.columns:
                    justify = "right" if col in ['Incrust.%', 'IMO', 'Dias*', 'Eventos', 'Penalidade'] else "left"
                    table.add_column(col, justify=justify)

                for _, row in df_validacao.iterrows():
                    status = row['Status']
                    if status == 'Crítica':
                        style = "bold red"
                    elif status == 'Pesada':
                        style = "red"
                    elif status == 'Moderada':
                        style = "yellow"
                    elif status == 'Leve':
                        style = "blue"
                    else:
                        style = "green"

                    table.add_row(*[str(val) for val in row], style=style)

                console.print(table)
            else:
                print("\n" + df_validacao.to_string(index=False))

            # Estatísticas de validação (por avaliação)
            incrust_values = df_validacao['Incrust.%'].values

            # Distribuição
            clean_count = (incrust_values < 1.0).sum()
            leve_count = ((incrust_values >= 1.0) & (incrust_values < 15.0)).sum()
            moderada_count = ((incrust_values >= 15.0) & (incrust_values < 40.0)).sum()
            pesada_count = ((incrust_values >= 40.0) & (incrust_values < 70.0)).sum()
            critica_count = (incrust_values >= 70.0).sum()
            total_val = len(incrust_values)



            # Contar avaliações críticas e urgentes
            aval_criticas = (incrust_values >= 70).sum()
            aval_urgentes = (incrust_values >= 40).sum()
            total_aval = len(incrust_values)




            # Salvar predições detalhadas
            df_pred_export = df_pred_val[['shipName', 'startGMTDate', 'fouling_percentage',
                                          'fouling_rating_imo']].copy()
            df_pred_export.to_csv('predicoes_validacao_detalhadas.csv', index=False)
            print_success("Predições detalhadas salvas em: predicoes_validacao_detalhadas.csv")

            print_success(f"\nValidação concluída! Arquivos gerados:")
            print_info(f"  1. {output_file}")
            print_info(f"  2. predicoes_validacao_detalhadas.csv")

        else:
            print_warning("Nenhum evento com dados AIS encontrado para validação")

    except FileNotFoundError as e:
        print_warning(f"Alguns arquivos de validação não foram encontrados: {e}")
        print_info("  Validação será pulada")
    except Exception as e:
        print_warning(f"Erro durante validação: {e}")
        print_info("  Validação será pulada")

else:
    print_warning(f"Pasta de validação não encontrada: {VALIDATION_PATH}")
    print_info("  Executando validação com dados principais...")

    try:
        # Usar dados principais para validação
        df_eventos_val = df_eventos.copy()
        df_consumo_val = df_consumo.copy()
        df_navios_val = df_navios.copy()
        df_ais_val = df_ais.copy()

        print_success(f"Dados principais carregados para validação:")
        print_info(f"  Eventos: {df_eventos_val.shape[0]} | AIS: {df_ais_val.shape[0]}")

        # Criar template com todos os navios
        navios_unicos = df_eventos_val['shipName'].unique()
        datas_recentes = df_eventos_val.groupby('shipName')['startGMTDate'].max().reset_index()
        df_resultado = pd.DataFrame({
            'Embarcação': datas_recentes['shipName'],
            'Data': datas_recentes['startGMTDate'],
            'Condição do Casco': ''
        })

        # Processar validação com dados principais
        # [O mesmo código de processamento será executado]

        # Pré-processar dados de validação
        df_eventos_val.columns = df_eventos_val.columns.str.strip()
        df_consumo_val.columns = df_consumo_val.columns.str.strip()
        df_ais_val.columns = df_ais_val.columns.str.strip()

        # Parse datetimes
        for c in ["startGMTDate", "endGMTDate"]:
            if c in df_eventos_val.columns:
                df_eventos_val[c] = pd.to_datetime(df_eventos_val[c], errors='coerce')

        # Renomear SESSION_ID
        if "SESSION_ID" in df_consumo_val.columns:
            df_consumo_val.rename(columns={"SESSION_ID": "sessionId"}, inplace=True)

        # Processar AIS
        if 'DATAHORA' in df_ais_val.columns:
            df_ais_val['DATETIME'] = pd.to_datetime(df_ais_val['DATAHORA'], errors='coerce')
        elif 'DataHora' in df_ais_val.columns:
            df_ais_val['DATETIME'] = pd.to_datetime(df_ais_val['DataHora'], errors='coerce')

        if 'VELOCIDADE' in df_ais_val.columns:
            df_ais_val['speed_kn'] = pd.to_numeric(df_ais_val['VELOCIDADE'], errors='coerce')
        elif 'speed' in df_ais_val.columns:
            df_ais_val['speed_kn'] = pd.to_numeric(df_ais_val['speed'], errors='coerce')

        if 'NOME' in df_ais_val.columns:
            df_ais_val['shipName_ais'] = df_ais_val['NOME'].astype(str)
        elif 'shipName' in df_ais_val.columns:
            df_ais_val['shipName_ais'] = df_ais_val['shipName'].astype(str)
        elif 'ARQUIVO_ORIGEM' in df_ais_val.columns:
            df_ais_val['shipName_ais'] = df_ais_val['ARQUIVO_ORIGEM'].str.replace('.csv', '').astype(str)

        # Agregar AIS por evento
        print_info("Processando eventos com dados principais...")
        df_events_ais_val = aggregate_ais_by_event(df_eventos_val, df_ais_val)

        if not df_events_ais_val.empty:
            # Criar features avançadas
            df_events_ais_val = create_advanced_features(df_events_ais_val)

            # Processar IWS e criar days_since_clean
            df_events_ais_val = process_iws_and_docking_data(df_iws, df_eventos_val, df_events_ais_val)

            # Criar estágio de fouling
            def get_stage(days):
                if pd.isna(days):
                    return 2
                if days < 14:
                    return 0
                elif days < 42:
                    return 1
                elif days < 90:
                    return 2
                else:
                    return 3

            df_events_ais_val['fouling_stage'] = df_events_ais_val['days_since_clean'].apply(get_stage)

            # Criar biofouling risk score
            df_events_ais_val['biofouling_risk_score'] = (
                0.4 * (df_events_ais_val['days_since_clean'].fillna(90) / 180).clip(0, 1) +
                0.25 * (df_events_ais_val['velocity_risk'] / 3) +
                0.2 * df_events_ais_val['idle_time_ratio'] +
                0.15 * df_events_ais_val['temp_risk']
            ).clip(0, 1)

            # Merge com consumo
            if 'sessionId' in df_consumo_val.columns:
                df_cons_sum = df_consumo_val.groupby('sessionId', as_index=False)['CONSUMED_QUANTITY'].sum()
                df_events_ais_val = df_events_ais_val.merge(df_cons_sum, on='sessionId', how='left')

            # Preparar features para predição
            features_missing = [f for f in features_available if f not in df_events_ais_val.columns]
            for feat in features_missing:
                df_events_ais_val[feat] = 0

            df_pred_val = df_events_ais_val[features_available + ['shipName', 'startGMTDate']].copy()
            df_pred_val[features_available] = df_pred_val[features_available].fillna(0)

            X_val = df_pred_val[features_available].values

            # Fazer predições com ensemble
            predictions_val = {}
            for name, model in models.items():
                pred = model.predict(X_val)
                predictions_val[name] = pred

            # Ensemble com pesos
            y_pred_val_ensemble = sum(predictions_val[name] * weight for name, weight in zip(models.keys(), weights))

            df_pred_val['fouling_percentage'] = y_pred_val_ensemble

            # Converter para IMO
            def percentage_to_imo_rating(pct):
                if pd.isna(pct):
                    return np.nan
                if pct < 0.5:
                    return 0.0
                elif pct < 1.0:
                    return 0.5 + (pct - 0.5)
                elif pct < 15.0:
                    return 1.0 + (pct - 1.0) / 14.0
                elif pct < 40.0:
                    return 2.0 + (pct - 15.0) / 25.0
                elif pct < 70.0:
                    return 3.0 + (pct - 40.0) / 30.0
                else:
                    return 4.0

            df_pred_val['fouling_rating_imo'] = df_pred_val['fouling_percentage'].apply(percentage_to_imo_rating)

            # Preencher template
            resultados_preenchidos = []

            for idx, row in df_resultado.iterrows():
                navio = str(row.get('Embarcação', '')).strip()
                data_alvo = row.get('Data')

                if pd.isna(data_alvo) or navio == '' or navio == 'nan':
                    continue

                # Buscar predições para este navio
                mask = df_pred_val['shipName'].str.upper() == navio.upper()
                if not mask.any():
                    mask = df_pred_val['shipName'].str.upper().str.contains(navio.upper(), na=False)

                preds_navio = df_pred_val[mask].copy()

                if preds_navio.empty:
                    continue

                # Encontrar predição mais próxima
                preds_navio['time_diff'] = (preds_navio['startGMTDate'] - data_alvo).abs()
                closest = preds_navio.loc[preds_navio['time_diff'].idxmin()]
                pct = closest['fouling_percentage']
                imo = closest['fouling_rating_imo']

                # Classificar
                if pct < 1.0:
                    classe = "Limpo/Micro"
                elif pct < 15.0:
                    classe = "Leve"
                elif pct < 40.0:
                    classe = "Moderada"
                elif pct < 70.0:
                    classe = "Pesada"
                else:
                    classe = "Crítica"

                condicao = f"{pct:.1f}% ({classe}, IMO {imo:.2f})"
                data_formatada = data_alvo.strftime('%d/%m/%Y') if pd.notna(data_alvo) else ''

                resultados_preenchidos.append({
                    'Embarcação': navio,
                    'Data': data_formatada,
                    'Condição do Casco': condicao
                })

            df_resultado_final = pd.DataFrame(resultados_preenchidos)

            # Salvar resultado
            output_file = 'RESULTADO_Validacao_Preenchido.xlsx'
            df_resultado_final.to_excel(output_file, index=False)
            print_success(f"Resultado salvo em: {output_file}")

            # Salvar predições detalhadas
            df_pred_export = df_pred_val[['shipName', 'startGMTDate', 'fouling_percentage',
                                          'fouling_rating_imo']].copy()
            df_pred_export.to_csv('predicoes_validacao_detalhadas.csv', index=False)
            print_success("Predições detalhadas salvas em: predicoes_validacao_detalhadas.csv")

        else:
            print_warning("Nenhum evento com dados AIS encontrado")

    except Exception as e:
        print_warning(f"Erro durante validação com dados principais: {e}")
        import traceback
        traceback.print_exc()

print_header("SCRIPT CONCLUÍDO")

VALIDAÇÃO COM NAVIOS DE TESTE

─────────────────────────────

Carregando dados de validação...

✓   Consumo de validação: 7613 registros

✓   Dados navios de validação: 2 registros

...

✓ 
Validação concluída! Arquivos gerados:

1. RESULTADO_Validacao_Preenchido.xlsx

2. predicoes_validacao_detalhadas.csv

SCRIPT CONCLUÍDO

────────────────